# Análise de Homologias por BLAST

Com o objetivo de procurar genes homólogos aos genes de interesse e caracterizar funcionalmente os genes selecionados, recorreu-se ao BLAST (uma ferramenta de alinhamento local) para procurar sequências homólogas significativas em bases de dados, que, posteriorimente, serão utilizadas para identificar padrões consistentes ao nível da função.

Através do Biopython, foi feito um BLASTp para cada proteína correspondente ao gene de interesse com posterior seleção dos 3 melhores alinhamentos, avaliados de acordo com o e-value, percent identity e coverage, tendo sido definido como threshold 0.001, 90 e 90 para os valores destes, respetivamente. 

Dado que os genes/proteínas de interesse são de origem humana, um dos objetivos desta análise de homologias foi verificar a que espécies pertenciam as sequências homólogas escolhidas anteriormente, excluindo da lista compostos sintéticos, para analisar uma possível conservação destas proteínas de interesse entre diferentes espécies.

In [1]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW 
from Bio import SeqIO

import re

In [2]:
def BLAST_prot (protein_name):
    """
    Função que realiza o BLASTp para a proteína fornecida a a partir do ficheiro fasta desta
    """
    protein = SeqIO.read(open(f'{protein_name}_protein.fasta'), format="fasta") 
    result = NCBIWWW.qblast("blastp", "swissprot", protein.seq)
    with open(f'{protein_name}_protein.xml', "w") as save_file:
        save_file.write(result.read())
    save_file.close()
    result.close()


In [3]:
BLAST_prot('IL10')

In [4]:
BLAST_prot('STAT6')

In [5]:
BLAST_prot('TGFB1')

In [6]:
BLAST_prot('IL13')

In [7]:
def first_alignment (protein_name):
    result_handle = open(f'{protein_name}_protein.xml')
    blast_record = NCBIXML.read(result_handle)

    first = blast_record.alignments[0]
    print("FIRST ALIGNMENT: ", "\nAcession:" + first.accession, "\nHit id:" + first.hit_id)
    print("Definition: " + first.hit_def, "\nAlignment lenght: " , first.length, "\nNumber of HPSs: " , len(first.hsps))
    result_handle.close()

In [9]:
def fliter_blast(protein_name, e_value_threshold, percent_identity_threshold, coverage_threshold):
    """ 
    Função para filtrar os melhores alinhamentos com base no e-value, percent identity e coverage
    Output: Nº de hits do blast, informação sobre os 3 melhores alinhamentos e especificação das espécies a que as 3 proteínas homologas pertencem
    """
    result_handle = open(f'{protein_name}_protein.xml')
    blast_record = NCBIXML.read(result_handle)
    
    print("PARAMETERS: \nDatabase: ", blast_record.database)
    print("Matriz: " + blast_record.matrix)
    print("Gap penalties: " , blast_record.gap_penalties)
    print('\nHits: ', len(blast_record.alignments))
    
    top_alignments = []
    for alignment in blast_record.alignments:
        if 'synthetic construct' not in alignment.title.lower():
            alignment_info = {
                'title': alignment.title,
                'length': alignment.length,
                'e_value': float('inf'),
                'percent_identity': 0.0,
                'coverage': 0.0,
                'query': '',
                'match': '',
                'sbjct': ''       }
            for hsp in alignment.hsps:
                if hsp.expect < alignment_info['e_value'] and \
                 (hsp.identities / hsp.align_length) * 100 >= percent_identity_threshold and \
                 (hsp.align_length / alignment.length) * 100 >= coverage_threshold:

                    alignment_info['e_value'] = hsp.expect
                    alignment_info['percent_identity'] = (hsp.identities / hsp.align_length) * 100
                    alignment_info['coverage'] = (hsp.align_length / alignment.length) * 100
                    alignment_info['query'] = hsp.query[0:75] + '...'
                    alignment_info['match'] = hsp.match[0:75] + '...'
                    alignment_info['sbjct'] = hsp.sbjct[0:75] + '...'

            if alignment_info['e_value'] < e_value_threshold:
                top_alignments.append(alignment_info)
                if alignment_info['coverage'] < coverage_threshold:
                    top_alignments.append(alignment_info)
                    if alignment_info['percent_identity'] < percent_identity_threshold:
                        top_alignments.append(alignment_info)
    for i, alignment_info in enumerate(top_alignments[:4]):
        print(f'****Alignment {i+1}****')
        print('Sequence: ', alignment_info['title'])
        print('Length: ', alignment_info['length'])
        print('E-value: ', alignment_info['e_value'])
        print('Percent identity: {:.2f}%'.format(alignment_info['percent_identity']))
        print('Coverage: {:.2f}%'.format(alignment_info['coverage']))
        print(alignment_info['query'])
        print(alignment_info['match'])
        print(alignment_info['sbjct'], '\n')

        species_list = []

    for alignment_info in top_alignments[:4]:
        title = alignment_info['title']
        match = re.search(r"\[(.*?)\]", title)
        if match:
            species = match.group(1)
            species_list.append(species)
    print("\nOrganisms:")
    for species in species_list:
        print(species)
        
    result_handle.close()
    

## IL10

In [10]:
first_alignment('IL10')

FIRST ALIGNMENT:  
Acession:P22301 
Hit id:sp|P22301.1|
Definition: RecName: Full=Interleukin-10; Short=IL-10; AltName: Full=Cytokine synthesis inhibitory factor; Short=CSIF; Flags: Precursor [Homo sapiens] 
Alignment lenght:  178 
Number of HPSs:  1


Após o BLASTp da proteína humana IL10, a primeira proteína da lista de sequências possivelmente homólogas é, como esperado, a proteína IL10 em Homo Sapiens.

In [11]:
e_value_threshold = 0.001
percent_identity_threshold = 90
coverage_threshold = 90 
fliter_blast('IL10', e_value_threshold, percent_identity_threshold, coverage_threshold)

PARAMETERS: 
Database:  swissprot
Matriz: BLOSUM62
Gap penalties:  (11, 1)

Hits:  30
****Alignment 1****
Sequence:  sp|P22301.1| RecName: Full=Interleukin-10; Short=IL-10; AltName: Full=Cytokine synthesis inhibitory factor; Short=CSIF; Flags: Precursor [Homo sapiens]
Length:  178
E-value:  1.99301e-132
Percent identity: 100.00%
Coverage: 100.00%
MHSSALLCCLVLLTGVRASPGQGTQSENSCTHFPGNLPNMLRDLRDAFSRVKTFFQMKDQLDNLLLKESLLEDFK...
MHSSALLCCLVLLTGVRASPGQGTQSENSCTHFPGNLPNMLRDLRDAFSRVKTFFQMKDQLDNLLLKESLLEDFK...
MHSSALLCCLVLLTGVRASPGQGTQSENSCTHFPGNLPNMLRDLRDAFSRVKTFFQMKDQLDNLLLKESLLEDFK... 

****Alignment 2****
Sequence:  sp|A2T6Z6.1| RecName: Full=Interleukin-10; Short=IL-10; AltName: Full=Cytokine synthesis inhibitory factor; Short=CSIF; Flags: Precursor [Pan troglodytes]
Length:  178
E-value:  3.61267e-131
Percent identity: 98.88%
Coverage: 100.00%
MHSSALLCCLVLLTGVRASPGQGTQSENSCTHFPGNLPNMLRDLRDAFSRVKTFFQMKDQLDNLLLKESLLEDFK...
MHSSALLCCLVLLTGVRASPGQGTQSENSCTHFPGNLPNMLRDLRDAFSRVKTFFQMKDQLDNLLLKE

Após a filtragem dos resultados do BLASTp, como esperado, o 1º alinhamento continua a corresponder à sequência da proteína IL10 em Homo Sapiens. Os restantes três alinhamentos correspondem à proteína IL10 encontrada em Pan troglodytes (chimpanzé), Macaca fascicularis (Macaco cinomolgos) e Macaca mulatta (macaco-rhesus), sugerindo uma grande conservação da proteína dado à homologia encontrada em espécies relacionadas como é o caso destas.

## STAT6

In [14]:
first_alignment('STAT6')

FIRST ALIGNMENT:  
Acession:P42226 
Hit id:sp|P42226.1|
Definition: RecName: Full=Signal transducer and activator of transcription 6; AltName: Full=IL-4 Stat [Homo sapiens] 
Alignment lenght:  847 
Number of HPSs:  1


In [15]:
fliter_blast('STAT6', e_value_threshold, percent_identity_threshold, coverage_threshold)

PARAMETERS: 
Database:  swissprot
Matriz: BLOSUM62
Gap penalties:  (11, 1)

Hits:  40
****Alignment 1****
Sequence:  sp|P42226.1| RecName: Full=Signal transducer and activator of transcription 6; AltName: Full=IL-4 Stat [Homo sapiens]
Length:  847
E-value:  0.0
Percent identity: 100.00%
Coverage: 100.00%
MSLWGLVSKMPPEKVQRLYVDFPQHLRHLLGDWLESQPWEFLVGSDAFCCNLASALLSDTVQHLQASVGEQGEGS...
MSLWGLVSKMPPEKVQRLYVDFPQHLRHLLGDWLESQPWEFLVGSDAFCCNLASALLSDTVQHLQASVGEQGEGS...
MSLWGLVSKMPPEKVQRLYVDFPQHLRHLLGDWLESQPWEFLVGSDAFCCNLASALLSDTVQHLQASVGEQGEGS... 


Organisms:
Homo sapiens


In [16]:
fliter_blast('STAT6', e_value_threshold, 80, 80)

PARAMETERS: 
Database:  swissprot
Matriz: BLOSUM62
Gap penalties:  (11, 1)

Hits:  40
****Alignment 1****
Sequence:  sp|P42226.1| RecName: Full=Signal transducer and activator of transcription 6; AltName: Full=IL-4 Stat [Homo sapiens]
Length:  847
E-value:  0.0
Percent identity: 100.00%
Coverage: 100.00%
MSLWGLVSKMPPEKVQRLYVDFPQHLRHLLGDWLESQPWEFLVGSDAFCCNLASALLSDTVQHLQASVGEQGEGS...
MSLWGLVSKMPPEKVQRLYVDFPQHLRHLLGDWLESQPWEFLVGSDAFCCNLASALLSDTVQHLQASVGEQGEGS...
MSLWGLVSKMPPEKVQRLYVDFPQHLRHLLGDWLESQPWEFLVGSDAFCCNLASALLSDTVQHLQASVGEQGEGS... 

****Alignment 2****
Sequence:  sp|P52633.2| RecName: Full=Signal transducer and transcription activator 6 [Mus musculus]
Length:  837
E-value:  0.0
Percent identity: 85.60%
Coverage: 101.19%
MSLWGLVSKMPPEKVQRLYVDFPQHLRHLLGDWLESQPWEFLVGSDAFCCNLASALLSDTVQHLQASVGEQGEGS...
MSLWGL+SKM PEK+QRLYVDFPQ LRHLL DWLESQPWEFLVGSDAFC N+ASALLS TVQ LQA+ GEQG+G+...
MSLWGLISKMSPEKLQRLYVDFPQRLRHLLADWLESQPWEFLVGSDAFCYNMASALLSATVQRLQATAGEQGKGN... 


Organisms:
Homo sapiens


Após a filtragem dos resultados do BLASTp, como esperado, o 1º resultado corresponde à sequência da proteína STAT6 em Homo Sapiens. Porém, mantendo os valores threshold de e-value, percent identity e coverage definidos para as outras proteínas, não é possível encontrar mais nenhuma homologia, o que indica que não há uma homologia/semelhança tão grande como nas outras proteínas de interesse. Se reduzirmos os thresholds de percentage identity e coverage para 80, é identificada uma sequência homóloga em Mus musculus, sugerindo que não há uma grande conservação da STAT6 em diferentes espécies.

## IL13

In [17]:
first_alignment('IL13')

FIRST ALIGNMENT:  
Acession:P35225 
Hit id:sp|P35225.2|
Definition: RecName: Full=Interleukin-13; Short=IL-13; Flags: Precursor [Homo sapiens] 
Alignment lenght:  146 
Number of HPSs:  1


In [18]:
fliter_blast('IL13', e_value_threshold, percent_identity_threshold, coverage_threshold)

PARAMETERS: 
Database:  swissprot
Matriz: BLOSUM62
Gap penalties:  (11, 1)

Hits:  10
****Alignment 1****
Sequence:  sp|P35225.2| RecName: Full=Interleukin-13; Short=IL-13; Flags: Precursor [Homo sapiens]
Length:  146
E-value:  8.68875e-104
Percent identity: 99.32%
Coverage: 100.00%
MHPLLNPLLLALGLMALLLTTVIALTCLGGFASPGPVPPSTALRELIEELVNITQNQKAPLCNGSMVWSINLTAG...
MHPLLNPLLLALGLMALLLTTVIALTCLGGFASPGPVPPSTALRELIEELVNITQNQKAPLCNGSMVWSINLTAG...
MHPLLNPLLLALGLMALLLTTVIALTCLGGFASPGPVPPSTALRELIEELVNITQNQKAPLCNGSMVWSINLTAG... 

****Alignment 2****
Sequence:  sp|P61126.1| RecName: Full=Interleukin-13; Short=IL-13; Flags: Precursor [Pan troglodytes]
Length:  132
E-value:  3.16043e-93
Percent identity: 98.48%
Coverage: 100.00%
MALLLTTVIALTCLGGFASPGPVPPSTALRELIEELVNITQNQKAPLCNGSMVWSINLTAGMYCAALESLINVSG...
MALLLTTVIALTCLGGFASPGPVPPSTALRELIEELVNITQNQKAPLCNGSMVWSINLTAG+YCAALESLINVSG...
MALLLTTVIALTCLGGFASPGPVPPSTALRELIEELVNITQNQKAPLCNGSMVWSINLTAGVYCAALESLINVSG... 

****Alignment 3****
Sequence:  sp|Q5I6

Após a filtragem dos resultados do BLASTp, como esperado, o 1º resultado corresponde à sequência da proteína IL13 em Homo Sapiens. Os restantes três alinhamentos correspondem à proteína encontrada em Pan troglodytes (chimpanzé), Macaca thibetana (Macaco Tibetano) e Macaca mulatta (macaco-rhesus), sugerindo uma conservação da proteína dado à homologia encontrada em espécies relacionadas como é o caso destas.

## TGFβ1

In [19]:
first_alignment('TGFB1')

FIRST ALIGNMENT:  
Acession:P01137 
Hit id:sp|P01137.3|
Definition: RecName: Full=Transforming growth factor beta-1 proprotein; Contains: RecName: Full=Latency-associated peptide; Short=LAP; Contains: RecName: Full=Transforming growth factor beta-1; Short=TGF-beta-1; Flags: Precursor [Homo sapiens] 
Alignment lenght:  390 
Number of HPSs:  1


In [20]:
fliter_blast('TGFB1', e_value_threshold, percent_identity_threshold, coverage_threshold)

PARAMETERS: 
Database:  swissprot
Matriz: BLOSUM62
Gap penalties:  (11, 1)

Hits:  50
****Alignment 1****
Sequence:  sp|P01137.3| RecName: Full=Transforming growth factor beta-1 proprotein; Contains: RecName: Full=Latency-associated peptide; Short=LAP; Contains: RecName: Full=Transforming growth factor beta-1; Short=TGF-beta-1; Flags: Precursor [Homo sapiens]
Length:  390
E-value:  0.0
Percent identity: 100.00%
Coverage: 100.00%
MPPSGLRLLPLLLPLLWLLVLTPGRPAAGLSTCKTIDMELVKRKRIEAIRGQILSKLRLASPPSQGEVPPGPLPE...
MPPSGLRLLPLLLPLLWLLVLTPGRPAAGLSTCKTIDMELVKRKRIEAIRGQILSKLRLASPPSQGEVPPGPLPE...
MPPSGLRLLPLLLPLLWLLVLTPGRPAAGLSTCKTIDMELVKRKRIEAIRGQILSKLRLASPPSQGEVPPGPLPE... 

****Alignment 2****
Sequence:  sp|P09533.1| RecName: Full=Transforming growth factor beta-1 proprotein; Contains: RecName: Full=Latency-associated peptide; Short=LAP; Contains: RecName: Full=Transforming growth factor beta-1; Short=TGF-beta-1; Flags: Precursor [Chlorocebus aethiops]
Length:  390
E-value:  0.0
Percent identity:

O 1º alinhamento da lista filtrada dos resultados do BLASTp é a pro-proteína TGFβ1, pecursor de TGFβ1, o que é expectável. Os restantes três resultados correspondem à proteína encontrada em Chlorocebus aethiops (conhecido como Macaco do Velho Mundo), Mustela putorius furo (Furão) e Bos taurus (Gado bovino), sugerindo que existe conservação da proteína em diferentes espécies.